In [ ]:
!nvidia-smi

Thu Jun 24 10:12:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/MayaChallenge/

# Эта библиотека нужна для чтения спутниковых снимков -- у них особый формат файла.
!easy_install GDAL

# !unzip ./DiscoverMayaChallenge_data.zip -d ./data

Mounted at /content/drive
/content/drive/MyDrive/MayaChallenge
Searching for GDAL
Best match: GDAL 2.2.2
Adding GDAL 2.2.2 to easy-install.pth file

Using /usr/lib/python2.7/dist-packages
Processing dependencies for GDAL
Finished processing dependencies for GDAL


In [ ]:
from segmentation_transforms import Compose, RandomHorizontalFlip, RandomCrop, Normalize, RandomResize

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import os

from osgeo import gdal, gdal_array

from pathlib import Path
from copy import copy
from torch.utils.data import Dataset, random_split
import torch
from torch import nn
from torchvision import transforms as T
from torch.nn import functional as F
from segmentation_transforms import Compose, RandomHorizontalFlip, RandomCrop, Normalize, RandomResize
import torchvision.transforms.functional as transforms_F

import pickle

from pathlib import Path

from chactun_dataset import ChactunDataset, UpsampleSentinelToLidar

mask_train_path = Path('./data/train_masks')
lidar_train_path = Path('./data/lidar_train')
sent1_train_path = Path('./data/Sentinel1_train')

sentinel bands выставлены на None чтобы получить значения по умолчанию

In [ ]:
class DotDict(dict):
    def __getattr__(self, attr):
        return self.get(attr)

    def __setattr__(self, key, value):
        self.__setitem__(key, value)

config = DotDict()
config.resize_min = 250
config.resize_max = 250
config.crop_size = 250
config.pretrained = True
config.num_classes = len(ChactunDataset.classes)
config.batch_size = 16
config.epochs = 10
config.lr = 1e-3
config.momentum = 0.9
config.sentinel1_bands = None # set default bands
config.sentinel2_bands = None

In [ ]:
!ls data

lidar_test   Sentinel1_test   Sentinel2_test   train_masks
lidar_train  Sentinel1_train  Sentinel2_train


In [ ]:
def get_transofrms(h_flip_prob, resize_min, resize_max, crop_size, mean, std):
    train_transform = Compose([        
        UpsampleSentinelToLidar(),
        RandomResize(resize_min, resize_max),
        RandomCrop(crop_size)
    ])
    test_transform = Compose([
        UpsampleSentinelToLidar()
    ])

    return train_transform, test_transform

def get_dataset(config, root='./data', val_size=0.25):
    train_transform, test_transform = get_transofrms(
        config.h_flip_prob,
        config.resize_min,
        config.resize_max,
        config.crop_size,
        config.mean,
        config.std
    )

    ds = ChactunDataset(root, is_train=True, transform=train_transform,
                        sentinel1_bands=config.sentinel1_bands,
                        sentinel2_bands=config.sentinel2_bands)
    train_ds, val_ds = random_split(ds, [len(ds) - int(len(ds) * val_size), 
                                        int(len(ds) * val_size)])

    test_ds = ChactunDataset(root, is_train=False, transform=test_transform,
                             sentinel1_bands=config.sentinel1_bands,
                             sentinel2_bands=config.sentinel2_bands)

    return train_ds, val_ds, test_ds

In [ ]:
from torch.utils.data import DataLoader

train_ds, val_ds, test_ds = get_dataset(config, './data')

train_dl = DataLoader(train_ds, batch_size=config.batch_size, shuffle=True, drop_last=True)
val_dl = DataLoader(val_ds, batch_size=config.batch_size, shuffle=True, drop_last=True)
test_dl = DataLoader(test_ds, batch_size=config.batch_size, drop_last=True)

Новые каналы добавлены, мы видим что первая размерность 8

In [ ]:
train_ds[0][0].shape

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:387: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


torch.Size([8, 250, 250])

# Модель

Использовалось 3 baseline модели, которые затем были ансамблированы с baseline моделью с помощью среднего геометрического их предсказаний.

In [ ]:
from torchvision import models

class DeepLabV3(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.model = models.segmentation.deeplabv3_resnet101(
            pretrained=self.config.pretrained, progress=True
        )

        self.model.classifier[4] = nn.Conv2d(256, self.config.num_classes, 1)

    def forward(self, X):
        return torch.sigmoid(self.model(X)['out'])

In [ ]:
from torch import optim

# Стандартное определение лосса, оптимизатора и lr scheduler.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
crit = nn.BCELoss()

In [ ]:
def discretize_segmentation_maps(probs, thresh):
    if thresh is None:
        thresh = [0.5, 0.5, 0.5]
    if isinstance(thresh, int):
        thresh = [thresh] * 3
    thresh = torch.from_numpy(np.array(thresh)).to(probs.device)
    return probs > thresh[:, None, None]

def get_ious(y_pred, y_true, thresh=None, eps=1e-7):
    y_pred = discretize_segmentation_maps(y_pred, thresh).float()
    y_true = y_true.float()
    with torch.no_grad():
        intersection = torch.sum(y_true * y_pred, dim=[2, 3])
        union = torch.sum(y_true, dim=[2, 3]) + torch.sum(y_pred, dim=[2, 3]) - intersection
        ious = ((intersection + + eps) / (union + eps)).mean(dim=0)
    return ious

In [ ]:
!pip install wandb

import wandb
wandb.init(config=config)

## Обучение

Объявим функцию обучения, которая будет по-отдельности дообучать модели с разными комбинациями каналов. Возьмём baseline решение, а также обучим 3 модели, каждая из которых имеет 1 из каналов лидара, а также 2 спутниковых. Наличие одного канала лидара даст хороший результат, так как он содержит общую информацию с двуми другими.

In [ ]:
# RGB=0..2, sentinel1=3..6, sentinel2=7
channels = [[0,3,7],
            [1,4,7],
            [2,5,7]]

In [ ]:
saved_models = []

In [ ]:
def produce_model():
    model = DeepLabV3(config)
    optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=config.momentum)
    sched = optim.lr_scheduler.OneCycleLR(optimizer, config.lr, 
                                        epochs=config.epochs, 
                                        steps_per_epoch=len(train_dl))
    wandb.watch(model, log_freq=100)
    model = model.to(device)
    
    return model, optimizer, sched

In [ ]:
from tqdm.notebook import tqdm, trange

def train(model_idx):
    cns = channels[model_idx]
    model, optimizer, sched = produce_model()

    for i in trange(config.epochs):
        model.train()
        for X, y in tqdm(train_dl):
            X = X[:,cns,:,:] # select 3 channels
            X = X.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            pred = model(X)
            loss = crit(pred, y)
            loss.backward()
            optimizer.step()
            sched.step()

            #LOG metrics to wandb
            ious = get_ious(pred, y, config.prediction_thresh).cpu()
            metrics = {'iou_' + class_name: iou_score.item()
                    for class_name, iou_score in zip(ChactunDataset.classes, ious)}
            metrics['avg_iou'] = ious.mean()
            metrics['loss'] = loss.item()
            metrics['lr'] = sched.get_last_lr()
            wandb.log(metrics)
        model.eval()
        losses = []
        ious = []
        for X, y in val_dl:
            X = X[:,cns,:,:]
            X = X.to(device)
            y = y.to(device)

            with torch.no_grad():
                pred = model(X)
                losses.append(crit(pred, y).item())
                iou = get_ious(pred, y, config.prediction_thresh)
                ious.append(iou.cpu().numpy())
        metrics = {'val_iou_' + class_name: iou_score.item()
                    for class_name, iou_score in zip(ChactunDataset.classes, np.stack(ious).mean(axis=0))}
        metrics['val_avg_iou'] = np.array(ious).mean(axis=0).mean()
        metrics['val_loss'] = np.mean(losses)
        wandb.log(metrics)
    model = model.cpu()
    return model

In [ ]:
m0 = train(0)
torch.save(m0.state_dict(), 'm0_weights-10e.pth')

In [ ]:
m1 = train(1)
torch.save(m1.state_dict(), 'm1_weights-10e.pth')

In [ ]:
m2 = train(2)
torch.save(m2.state_dict(), 'm2_weights-10e.pth')

In [ ]:
wandb.run.finish()

## Оценка модели

Загрузим 3 модели обученные на различных каналах, а также RGB модель. Используя геометрическое среднее получим предсказания сборной модели.





In [ ]:
def geom_avg(model_preds):
    N = len(model_preds)
    result = torch.ones_like(model_preds[0])
    for pred in model_preds:
        result *= pred
    return result.pow(1/N)

In [ ]:
m0 = DeepLabV3(config)
m0.load_state_dict(torch.load('m0_weights.pth'))

m1 = DeepLabV3(config)
m1.load_state_dict(torch.load('m1_weights.pth'))

m2 = DeepLabV3(config)
m2.load_state_dict(torch.load('m2_weights.pth'))

mRGB = DeepLabV3(config)
mRGB.load_state_dict(torch.load('model_weights-9.pth'))

ensemble_models = [m0, m1, m2, mRGB]

Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth


In [ ]:
def ensemble_pred(models, X):
    pred_list = []
    for model_idx, model in enumerate(models):
        cns = [0, 1, 2]
        if model_idx < 3:
            cns = channels[model_idx]

        model.eval()
        model = model.to(device)
        pred = model(X[:,cns,:,:]).detach()
        pred_list.append(pred)
        model.cpu()
    pred = geom_avg(pred_list)
    return pred

In [ ]:
X,y = next(iter(train_dl))

In [ ]:
ensemble_pred(ensemble_models, X.to(device)).mean()

## Проверка accuracy

In [ ]:
from tqdm.notebook import tqdm, trange

for i in trange(config.epochs):
    losses = []
    ious = []
    for X, y in tqdm(val_dl):
        X = X.to(device)
        y = y.to(device)

        with torch.no_grad():
            pred = ensemble_pred(ensemble_models, X)
            losses.append(crit(pred, y).item())
            iou = get_ious(pred, y, config.prediction_thresh)
            ious.append(iou.cpu().numpy())

    metrics = {'val_iou_' + class_name: iou_score.item()
                for class_name, iou_score in zip(ChactunDataset.classes, np.stack(ious).mean(axis=0))}
    metrics['val_avg_iou'] = np.array(ious).mean(axis=0).mean()
    metrics['val_loss'] = np.mean(losses)

    wandb.log(metrics)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:387: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
wandb.run.finish()